In [1]:
import itertools

In [2]:
import numpy as np, pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def heuristic(C):
    S = C.copy()
    S.loc[:, :] = -np.infty
    for i, j in itertools.product(C.index, C.columns):
        if C.loc[i,j].round(2) != 0:
            S.loc[i,j] = -np.infty
            continue
        mr, mc = 2*[np.infty]
        for i1 in C.index:
            if i1 == i: continue
            mc = min(mc, C.loc[i1, j])
        for j1 in C.columns:
            if j1 == j: continue
            mr = min(mr, C.loc[i, j1])
        S.loc[i,j] = mr + mc
    return S

In [4]:
def row_transform(C):
    m = C.min(axis=1)
    return C.sub(m, axis='index'), m

def col_transform(C):
    m = C.min(axis=0)
    return C.sub(m), m

In [5]:
N=6
C = np.array([
    [7+5*np.abs(np.sin(i+1)*np.cos(1+j)) for j in range(N)]
    for i in range(N)
])
C = C.round(2)
C = pd.DataFrame(C, index=[1+i for i in range(N)], columns=[1+i for i in range(N)])
C

,1,2,3,4,5,6
1,9.27,8.75,11.17,9.75,8.19,11.04
2,9.46,8.89,11.50,9.97,8.29,11.37
3,7.38,7.29,7.70,7.46,7.20,7.68
4,9.04,8.57,10.75,9.47,8.07,10.63
5,9.59,9.00,11.75,10.13,8.36,11.60
6,7.75,7.58,8.38,7.91,7.40,8.34


In [6]:
for i in range(N):
    C.loc[1+i, 1+i] = np.infty
C

,1,2,3,4,5,6
1,inf,8.75,11.17,9.75,8.19,11.04
2,9.46,inf,11.50,9.97,8.29,11.37
3,7.38,7.29,inf,7.46,7.20,7.68
4,9.04,8.57,10.75,inf,8.07,10.63
5,9.59,9.00,11.75,10.13,inf,11.60
6,7.75,7.58,8.38,7.91,7.40,inf


In [7]:
C2, C2mr = row_transform(C)
C2 = C2.round(2)
C2

,1,2,3,4,5,6
1,inf,0.56,2.98,1.56,0.00,2.85
2,1.17,inf,3.21,1.68,0.00,3.08
3,0.18,0.09,inf,0.26,0.00,0.48
4,0.97,0.50,2.68,inf,0.00,2.56
5,0.59,0.00,2.75,1.13,inf,2.60
6,0.35,0.18,0.98,0.51,0.00,inf


In [8]:
C2mr

1   8.19
2   8.29
3   7.20
4   8.07
5   9.00
6   7.40
dtype: float64

In [9]:
C2, C2mc = col_transform(C2)
C2 = C2.round(2)
C2

,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,2.37
2,0.99,inf,2.23,1.42,0.00,2.60
3,0.00,0.09,inf,0.00,0.00,0.00
4,0.79,0.50,1.70,inf,0.00,2.08
5,0.41,0.00,1.77,0.87,inf,2.12
6,0.17,0.18,0.00,0.25,0.00,inf


In [10]:
C2mc

1   0.18
2   0.00
3   0.98
4   0.26
5   0.00
6   0.48
dtype: float64

In [11]:
l2r, l2c = C2mr.sum(), C2mc.sum()
l2 = l2r + l2c
l2r, l2c, l2

(48.15, 1.9, 50.05)

In [12]:
.18+.99+.87+1.70, .18+.99+.87+1.70+l2

(3.74, 53.79)

In [13]:
H2 = heuristic(C2)
H2

,1,2,3,4,5,6
1,-inf,-inf,-inf,-inf,0.56,-inf
2,-inf,-inf,-inf,-inf,0.99,-inf
3,0.17,-inf,-inf,0.25,0.00,2.08
4,-inf,-inf,-inf,-inf,0.50,-inf
5,-inf,0.50,-inf,-inf,-inf,-inf
6,-inf,-inf,1.70,-inf,0.00,-inf


In [14]:
B1u = 8.19+9+9.97+10.63+8.38+7.38
B1u, round(B1u, 2)

(53.550000000000004, 53.55)

In [15]:
C2

,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,2.37
2,0.99,inf,2.23,1.42,0.00,2.60
3,0.00,0.09,inf,0.00,0.00,0.00
4,0.79,0.50,1.70,inf,0.00,2.08
5,0.41,0.00,1.77,0.87,inf,2.12
6,0.17,0.18,0.00,0.25,0.00,inf


In [16]:
round(50.05+2.08, 2)

52.13

In [17]:
C_2_1 = C2.loc[[1,2,4,5,6], [1, 2, 3, 4, 5]]
# C_2_1 = C_2_1[:,np.array([1,2,3,4,5])-1]
C_2_1.loc[6, 3] = np.infty
C_2_1

,1,2,3,4,5
1,inf,0.56,2.00,1.30,0.00
2,0.99,inf,2.23,1.42,0.00
4,0.79,0.50,1.70,inf,0.00
5,0.41,0.00,1.77,0.87,inf
6,0.17,0.18,inf,0.25,0.00


In [18]:
C3, C3_mr = row_transform(C_2_1)
C3

,1,2,3,4,5
1,inf,0.56,2.00,1.30,0.00
2,0.99,inf,2.23,1.42,0.00
4,0.79,0.50,1.70,inf,0.00
5,0.41,0.00,1.77,0.87,inf
6,0.17,0.18,inf,0.25,0.00


In [19]:
C3_mr

1   0.00
2   0.00
4   0.00
5   0.00
6   0.00
dtype: float64

In [20]:
C3, C3_mc = col_transform(C3)
C3

,1,2,3,4,5
1,inf,0.56,0.30,1.05,0.00
2,0.82,inf,0.53,1.17,0.00
4,0.62,0.50,0.00,inf,0.00
5,0.24,0.00,0.07,0.62,inf
6,0.00,0.18,inf,0.00,0.00


In [21]:
C3_mc

1   0.17
2   0.00
3   1.70
4   0.25
5   0.00
dtype: float64

In [22]:
l3 = C3_mr.sum() + C3_mc.sum()
l3

2.12

In [23]:
round(l2 + l3, 2)

52.17

In [24]:
H3 = heuristic(C3)
H3

,1,2,3,4,5
1,-inf,-inf,-inf,-inf,0.30
2,-inf,-inf,-inf,-inf,0.53
4,-inf,-inf,0.07,-inf,0.00
5,-inf,0.25,-inf,-inf,-inf
6,0.24,-inf,-inf,0.62,0.00


In [25]:
52.17+.53

52.7

In [26]:
C4 = C3.loc[[1,4,5,6,], [1,2,3,4]]
C4.loc[5, 2] = np.infty
C4

,1,2,3,4
1,inf,0.56,0.30,1.05
4,0.62,0.50,0.00,inf
5,0.24,inf,0.07,0.62
6,0.00,0.18,inf,0.00


In [27]:
C4, C4mr = row_transform(C4)
print(C4mr)
C4

1   0.30
4   0.00
5   0.07
6   0.00
dtype: float64


,1,2,3,4
1,inf,0.26,0.00,0.75
4,0.62,0.50,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.18,inf,0.00


In [28]:
C4, C4mc = col_transform(C4)
print(C4mc)
C4

1   0.00
2   0.18
3   0.00
4   0.00
dtype: float64


,1,2,3,4
1,inf,0.08,0.00,0.75
4,0.62,0.32,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.00,inf,0.00


In [29]:
H4 = heuristic(C4)
H4

,1,2,3,4
1,-inf,-inf,0.08,-inf
4,-inf,-inf,0.32,-inf
5,-inf,-inf,0.17,-inf
6,0.17,0.08,-inf,0.55


In [30]:
L4 = C4mr.sum() + C4mc.sum()
L4

0.55

In [31]:
L4 + l3 + l2

52.72

In [32]:
round(52.72+.55, 2)

53.27

In [33]:
C5 = C4.loc[[1,4,5], [1,2,3]]
C5

,1,2,3
1,inf,0.08,0.00
4,0.62,0.32,0.00
5,0.17,inf,0.00


In [34]:
C5, C5mc = col_transform(C5)
C5

,1,2,3
1,inf,0.00,0.00
4,0.45,0.24,0.00
5,0.00,inf,0.00


In [35]:
C5mc.sum(), C5mc

(0.24999999999999992, 1   0.17
 2   0.08
 3   0.00
 dtype: float64)

In [36]:
L5 = C5mc.sum()
round(L5 + L4 + l3 + l2, 2)

52.97

In [37]:
H5 = heuristic(C5)
H5

,1,2,3
1,-inf,0.24,0.00
4,-inf,-inf,0.24
5,0.45,-inf,0.00


In [38]:
52.97 + .45

53.42

In [39]:
52.97+.32

53.29

In [40]:
C7 = C4.copy()
C7.loc[6, 4] = np.infty
C7

,1,2,3,4
1,inf,0.08,0.00,0.75
4,0.62,0.32,0.00,inf
5,0.17,inf,0.00,0.55
6,0.00,0.00,inf,inf


In [41]:
C7, C7mc = col_transform(C7)
print(C7mc)
C7

1   0.00
2   0.00
3   0.00
4   0.55
dtype: float64


,1,2,3,4
1,inf,0.08,0.00,0.20
4,0.62,0.32,0.00,inf
5,0.17,inf,0.00,0.00
6,0.00,0.00,inf,inf


In [42]:
H7 = heuristic(C7)
H7

,1,2,3,4
1,-inf,-inf,0.08,-inf
4,-inf,-inf,0.32,-inf
5,-inf,-inf,0.00,0.20
6,0.17,0.08,-inf,-inf


In [43]:
53.27+.32

53.59

In [44]:
C8 = C7.drop(6, axis=0).drop(3, axis=1)
C8

,1,2,4
1,inf,0.08,0.20
4,0.62,0.32,inf
5,0.17,inf,0.00


In [45]:
C8, C8m = col_transform(C8)
print(C8m)
C8

1   0.17
2   0.08
4   0.00
dtype: float64


,1,2,4
1,inf,0.00,0.20
4,0.45,0.24,inf
5,0.00,inf,0.00


In [46]:
heuristic(C8)

,1,2,4
1,-inf,0.44,-inf
4,-inf,-inf,-inf
5,0.45,-inf,0.20


In [47]:
53.27+.17+.08

53.52

In [48]:
C9 = C3.copy()
C9.loc[2, 5] = np.infty
C9

,1,2,3,4,5
1,inf,0.56,0.30,1.05,0.00
2,0.82,inf,0.53,1.17,inf
4,0.62,0.50,0.00,inf,0.00
5,0.24,0.00,0.07,0.62,inf
6,0.00,0.18,inf,0.00,0.00


In [49]:
C9, C9mr = row_transform(C9)
print(C9mr)
C9

1   0.00
2   0.53
4   0.00
5   0.00
6   0.00
dtype: float64


,1,2,3,4,5
1,inf,0.56,0.30,1.05,0.00
2,0.29,inf,0.00,0.64,inf
4,0.62,0.50,0.00,inf,0.00
5,0.24,0.00,0.07,0.62,inf
6,0.00,0.18,inf,0.00,0.00


In [50]:
52.17+.53

52.7

In [51]:
heuristic(C9)

,1,2,3,4,5
1,-inf,-inf,-inf,-inf,0.30
2,-inf,-inf,0.29,-inf,-inf
4,-inf,-inf,0.00,-inf,0.00
5,-inf,0.25,-inf,-inf,-inf
6,0.24,-inf,-inf,0.62,0.00


In [52]:
52.7+.62

53.32

In [53]:
C10 = C9.drop(6).drop(4, axis=1)
C10

,1,2,3,5
1,inf,0.56,0.30,0.00
2,0.29,inf,0.00,inf
4,0.62,0.50,0.00,0.00
5,0.24,0.00,0.07,inf


In [54]:
C10, C10mc = col_transform(C10)
print(C10mc)
C10

1   0.24
2   0.00
3   0.00
5   0.00
dtype: float64


,1,2,3,5
1,inf,0.56,0.30,0.00
2,0.05,inf,0.00,inf
4,0.38,0.50,0.00,0.00
5,0.00,0.00,0.07,inf


In [55]:
52.7+.53

53.230000000000004

In [56]:
heuristic(C10)

,1,2,3,5
1,-inf,-inf,-inf,0.30
2,-inf,-inf,0.05,-inf
4,-inf,-inf,0.00,0.00
5,0.05,0.50,-inf,-inf


In [57]:
C11 = C10.drop(5).drop(2, axis=1)
C11.loc[2, 5] = np.infty
C11

,1,3,5
1,inf,0.30,0.00
2,0.05,0.00,inf
4,0.38,0.00,0.00


In [58]:
C11, C11m = col_transform(C11)
print(C11m)
C11

1   0.05
3   0.00
5   0.00
dtype: float64


,1,3,5
1,inf,0.30,0.00
2,0.00,0.00,inf
4,0.33,0.00,0.00


In [59]:
heuristic(C11)

,1,3,5
1,-inf,-inf,0.30
2,0.33,0.00,-inf
4,-inf,0.00,0.00


In [60]:
53.23+.05

53.279999999999994

In [61]:
53.28+.33

53.61

In [62]:
C11.drop(2).drop(1, axis=1)

,3,5
1,0.30,0.00
4,0.00,0.00


In [63]:
C12 = C2.copy()
C12.loc[3, 6] = np.infty
C12

,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,2.37
2,0.99,inf,2.23,1.42,0.00,2.60
3,0.00,0.09,inf,0.00,0.00,inf
4,0.79,0.50,1.70,inf,0.00,2.08
5,0.41,0.00,1.77,0.87,inf,2.12
6,0.17,0.18,0.00,0.25,0.00,inf


In [64]:
C12, C12m = col_transform(C12)
print(C12m)
C12

1   0.00
2   0.00
3   0.00
4   0.00
5   0.00
6   2.08
dtype: float64


,1,2,3,4,5,6
1,inf,0.56,2.00,1.30,0.00,0.29
2,0.99,inf,2.23,1.42,0.00,0.52
3,0.00,0.09,inf,0.00,0.00,inf
4,0.79,0.50,1.70,inf,0.00,0.00
5,0.41,0.00,1.77,0.87,inf,0.04
6,0.17,0.18,0.00,0.25,0.00,inf


In [65]:
heuristic(C12)

,1,2,3,4,5,6
1,-inf,-inf,-inf,-inf,0.29,-inf
2,-inf,-inf,-inf,-inf,0.52,-inf
3,0.17,-inf,-inf,0.25,0.00,-inf
4,-inf,-inf,-inf,-inf,0.00,0.04
5,-inf,0.13,-inf,-inf,-inf,-inf
6,-inf,-inf,1.70,-inf,0.00,-inf


In [66]:
52.13+1.7

53.830000000000005

In [67]:
C13 = C12.drop(6).drop(3, axis=1)
C13.loc[3,6] = np.infty
C13

,1,2,4,5,6
1,inf,0.56,1.30,0.00,0.29
2,0.99,inf,1.42,0.00,0.52
3,0.00,0.09,0.00,0.00,inf
4,0.79,0.50,inf,0.00,0.00
5,0.41,0.00,0.87,inf,0.04


In [68]:
heuristic(C13)

,1,2,4,5,6
1,-inf,-inf,-inf,0.29,-inf
2,-inf,-inf,-inf,0.52,-inf
3,0.41,-inf,0.87,0.00,-inf
4,-inf,-inf,-inf,0.00,0.04
5,-inf,0.13,-inf,-inf,-inf


In [69]:
52.13+.87

53.0

In [70]:
C14 = C13.drop(3).drop(4, axis=1)
C14.loc[4, 1] = np.infty
C14

,1,2,5,6
1,inf,0.56,0.00,0.29
2,0.99,inf,0.00,0.52
4,inf,0.50,0.00,0.00
5,0.41,0.00,inf,0.04


In [71]:
C14, C14m = col_transform(C14)
C14

,1,2,5,6
1,inf,0.56,0.00,0.29
2,0.58,inf,0.00,0.52
4,inf,0.50,0.00,0.00
5,0.00,0.00,inf,0.04


In [72]:
print(C14m)

1   0.41
2   0.00
5   0.00
6   0.00
dtype: float64


In [73]:
heuristic(C14)

,1,2,5,6
1,-inf,-inf,0.29,-inf
2,-inf,-inf,0.52,-inf
4,-inf,-inf,0.00,0.04
5,0.58,0.50,-inf,-inf


In [74]:
52.13+.41

52.54

In [75]:
52.54+.58

53.12

In [76]:
C15 = C14.drop(5).drop(1, axis=1)
C15.loc[1,5] = np.infty
C15

,2,5,6
1,0.56,inf,0.29
2,inf,0.00,0.52
4,0.50,0.00,0.00


In [77]:
C15, C15mr = row_transform(C15)
print(C15mr)
display(C15)
C15, C15mc = col_transform(C15)
print(C15mc)
display(C15)

1   0.29
2   0.00
4   0.00
dtype: float64


,2,5,6
1,0.27,inf,0.00
2,inf,0.00,0.52
4,0.50,0.00,0.00


2   0.27
5   0.00
6   0.00
dtype: float64


,2,5,6
1,0.00,inf,0.00
2,inf,0.00,0.52
4,0.23,0.00,0.00


In [78]:
52.54+.29+.5

53.33

In [79]:
C16 = C14.copy()
C16.loc[5,1] = np.infty
C16

,1,2,5,6
1,inf,0.56,0.00,0.29
2,0.58,inf,0.00,0.52
4,inf,0.50,0.00,0.00
5,inf,0.00,inf,0.04


In [80]:
C16, C16mc = col_transform(C16)
print(C16mc)
C16

1   0.58
2   0.00
5   0.00
6   0.00
dtype: float64


,1,2,5,6
1,inf,0.56,0.00,0.29
2,0.00,inf,0.00,0.52
4,inf,0.50,0.00,0.00
5,inf,0.00,inf,0.04


In [81]:
heuristic(C16)

,1,2,5,6
1,-inf,-inf,0.29,-inf
2,inf,-inf,0.00,-inf
4,-inf,-inf,0.00,0.04
5,-inf,0.54,-inf,-inf


In [82]:
53.12+.54

53.66

In [83]:
C17 = C16.drop(5).drop(2, axis=1)
C17.loc[2,5] = np.infty
C17

,1,5,6
1,inf,0.00,0.29
2,0.00,inf,0.52
4,inf,0.00,0.00


In [84]:
53.12+.29

53.41

In [85]:
C18 = C13.copy()
C18.loc[3, 4] = np.infty
C18

,1,2,4,5,6
1,inf,0.56,1.30,0.00,0.29
2,0.99,inf,1.42,0.00,0.52
3,0.00,0.09,inf,0.00,inf
4,0.79,0.50,inf,0.00,0.00
5,0.41,0.00,0.87,inf,0.04


In [86]:
C18, C18mc = col_transform(C18)
print(C18mc)
C18

1   0.00
2   0.00
4   0.87
5   0.00
6   0.00
dtype: float64


,1,2,4,5,6
1,inf,0.56,0.43,0.00,0.29
2,0.99,inf,0.55,0.00,0.52
3,0.00,0.09,inf,0.00,inf
4,0.79,0.50,inf,0.00,0.00
5,0.41,0.00,0.00,inf,0.04


In [87]:
heuristic(C18)

,1,2,4,5,6
1,-inf,-inf,-inf,0.29,-inf
2,-inf,-inf,-inf,0.52,-inf
3,0.41,-inf,-inf,0.00,-inf
4,-inf,-inf,-inf,0.00,0.04
5,-inf,0.09,0.43,-inf,-inf


In [88]:
C19 = C18.drop(4).drop(1, axis=1)
C19.loc[1,4] = np.infty
C19

,2,4,5,6
1,0.56,inf,0.00,0.29
2,inf,0.55,0.00,0.52
3,0.09,inf,0.00,inf
5,0.00,0.00,inf,0.04


In [89]:
C19, C19mc = col_transform(C19)
print(C19mc)
C19

2   0.00
4   0.00
5   0.00
6   0.04
dtype: float64


,2,4,5,6
1,0.56,inf,0.00,0.25
2,inf,0.55,0.00,0.48
3,0.09,inf,0.00,inf
5,0.00,0.00,inf,0.00


In [90]:
heuristic(C19)

,2,4,5,6
1,-inf,-inf,0.25,-inf
2,-inf,-inf,0.48,-inf
3,-inf,-inf,0.09,-inf
5,0.09,0.55,-inf,0.25


In [91]:
C20 = C19.drop(5).drop(4, axis=1)
C20

,2,5,6
1,0.56,0.00,0.25
2,inf,0.00,0.48
3,0.09,0.00,inf


In [92]:
C20, C20mc = col_transform(C20)
print(C20mc)
C20

2   0.09
5   0.00
6   0.25
dtype: float64


,2,5,6
1,0.47,0.00,0.00
2,inf,0.00,0.23
3,0.00,0.00,inf


In [93]:
53.04+.09+.25

53.38

In [94]:
heuristic(C20)

,2,5,6
1,-inf,0.00,0.23
2,-inf,0.23,-inf
3,0.47,0.00,-inf


In [95]:
C21 = C20.drop(3).drop(2, axis=1)
C21

,5,6
1,0.00,0.00
2,0.00,0.23
